In [1]:
import pandas as pd
import pyarrow.parquet as pq
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import joblib
import numpy as np
import os
from scipy.sparse import save_npz, vstack, hstack

try:
    stopwords.words("english")
except LookupError:
    print("NLTK stopwords indiriliyor...")
    nltk.download("stopwords")

INPUT_PARQUET_PATH = "../data/raw/labeled_reviews.parquet"
OUTPUT_DIR = "../data/processed/"

TEXT_ONLY_DIR = os.path.join(OUTPUT_DIR, "text_data")
HYBRID_DIR = os.path.join(OUTPUT_DIR, "hybrid")
os.makedirs(TEXT_ONLY_DIR, exist_ok=True)
os.makedirs(HYBRID_DIR, exist_ok=True)

VECTORIZER_PATH = os.path.join(OUTPUT_DIR, "tfidf_vectorizer.joblib")

stop_words = set(stopwords.words("english"))
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

def create_behavioral_features(df):
    helpful_list = df["helpful"].apply(eval)
    df['helpful_votes'] = helpful_list.apply(lambda x: x[0])
    df['total_votes'] = helpful_list.apply(lambda x: x[1])
    df['helpfulness_ratio'] = df['helpful_votes'] / (df['total_votes'] + 0.001)

    df['text_length'] = df['full_text'].str.len()
    
    feature_df = df[['overall', 'helpfulness_ratio', 'text_length']].fillna(0)
    return feature_df.to_numpy()

print("Kurulum tamamlandı.")

Kurulum tamamlandı.


In [2]:
if not os.path.exists(VECTORIZER_PATH):
    print("Vektörleştirici oluşturuluyor...")
    parquet_file_vec = pq.ParquetFile(INPUT_PARQUET_PATH)
    sample_batch_vec = next(parquet_file_vec.iter_batches(batch_size=200000))
    df_sample = sample_batch_vec.to_pandas()
    df_sample["full_text"] = df_sample["summary"].astype(str) + " " + df_sample["reviewText"].astype(str)
    df_sample["cleaned_text"] = df_sample["full_text"].apply(clean_text)
    tfidf_vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1, 2))
    tfidf_vectorizer.fit(df_sample["cleaned_text"])
    joblib.dump(tfidf_vectorizer, VECTORIZER_PATH)
    print(f"Vektörleştirici '{VECTORIZER_PATH}' adresine kaydedildi.")
else:
    print("Vektörleştirici zaten mevcut, bu adım atlanıyor.")

Vektörleştirici zaten mevcut, bu adım atlanıyor.


In [3]:
all_reviewer_ids = pd.read_parquet(INPUT_PARQUET_PATH, columns=['reviewerID'])
reviewer_counts = all_reviewer_ids['reviewerID'].value_counts()

In [ ]:
import glob

INPUT_BASE_PATH = 'C:/Users/Aybars/.cache/kagglehub/datasets/naveedhn/amazon-product-review-spam-and-non-spam/versions/1'

tfidf_vectorizer = joblib.load(VECTORIZER_PATH)
json_files = glob.glob(os.path.join(INPUT_BASE_PATH, "*/*.json"))

for file in json_files:
    category_name = os.path.basename(os.path.dirname(file))
    category_output_dir = os.path.join(HYBRID_DIR, category_name)
    
    print(f"\n--- İşleniyor: {category_name} ---")
    final_files_check = [os.path.join(category_output_dir, f) for f in ["X_train.npz", "X_test.npz", "y_train.npy", "y_test.npy"]]
    if all(os.path.exists(p) for p in final_files_check):
        print(f"✅ '{category_name}' için dosyalar zaten mevcut, atlanıyor.")
        continue
    
    os.makedirs(category_output_dir, exist_ok=True)
    
    category_hybrid_chunks = []
    category_labels = []

    try:
        # Önce o kategoriye ait tüm etiketli veriyi bellekte biriktireceğiz
        category_chunks_list = []
        batch_iterator = pd.read_json(file, lines=True, chunksize=1000000)
        for df_chunk in batch_iterator:
            if "class" not in df_chunk.columns or df_chunk["class"].isnull().all():
                continue
            classified_chunk = df_chunk.dropna(subset=["class"]).copy()
            if not classified_chunk.empty:
                category_chunks_list.append(classified_chunk)

        if not category_chunks_list:
            print("\nBu kategoride etiketli veri bulunamadı.")
            continue
            
        # ⭐️ DEĞİŞİKLİK 2: Kategoriye ait tüm veriyi birleştirip yorumcu sayılarını hesaplıyoruz
        print(f"\n-> {category_name}: Tüm parçalar birleştirildi. Özellikler oluşturuluyor...")
        category_df = pd.concat(category_chunks_list, ignore_index=True)
        
        # Bu kategoriye özel yorumcu sayılarını (local counts) hesapla
        local_reviewer_counts = category_df['reviewerID'].value_counts()

        # Özellikleri oluşturma
        category_df['full_text'] = category_df['summary'].astype(str) + ' ' + category_df['reviewText'].astype(str)
        category_df['cleaned_text'] = category_df['full_text'].apply(clean_text)
        
        X_category_tfidf = tfidf_vectorizer.transform(category_df['cleaned_text'])
        
        # Güncellenmiş fonksiyonu çağır
        X_category_behavioral = create_behavioral_features(category_df, local_reviewer_counts)
        
        X_final_category = hstack([X_category_tfidf, X_category_behavioral])
        y_final_category = category_df['class'].to_numpy()

        # Kategoriye özel veriyi böl ve kaydet
        print("Veri setleri ayrılıyor ve kaydediliyor...")
        X_train, X_test, y_train, y_test = train_test_split(
            X_final_category, y_final_category, test_size=0.20, random_state=42, stratify=y_final_category
        )

        save_npz(os.path.join(category_output_dir, "X_train.npz"), X_train)
        save_npz(os.path.join(category_output_dir, "X_test.npz"), X_test)
        np.save(os.path.join(category_output_dir, "y_train.npy"), y_train)
        np.save(os.path.join(category_output_dir, "y_test.npy"), y_test)
        print(f"✅ '{category_name}' için {len(y_final_category)} satırlık veri seti oluşturuldu.")

    except Exception as e:
        print(f"\n🚨 HATA: {category_name} işlenirken bir sorun oluştu - {e}")

print("\n🚀 Tüm işlemler tamamlandı!")

Bir veya daha fazla veri seti eksik. Veri işleme adımları başlatılıyor...
-> Parça 1 işleniyor...
-> Parça 1 işleniyor...
-> Parça 1 işlendi
-> Parça 1 işlendi
-> Parça 2 işleniyor...
-> Parça 2 işleniyor...
-> Parça 2 işlendi
-> Parça 2 işlendi
-> Parça 3 işleniyor...
-> Parça 3 işleniyor...
-> Parça 3 işlendi
-> Parça 3 işlendi
-> Parça 4 işleniyor...
-> Parça 4 işleniyor...
-> Parça 4 işlendi
-> Parça 4 işlendi
-> Parça 5 işleniyor...
-> Parça 5 işleniyor...
-> Parça 5 işlendi
-> Parça 5 işlendi
-> Parça 6 işleniyor...
-> Parça 6 işleniyor...
-> Parça 6 işlendi
-> Parça 6 işlendi
-> Parça 7 işleniyor...
-> Parça 7 işleniyor...
-> Parça 7 işlendi
-> Parça 7 işlendi
-> Parça 8 işleniyor...
-> Parça 8 işleniyor...
-> Parça 8 işlendi
-> Parça 8 işlendi
-> Parça 9 işleniyor...
-> Parça 9 işleniyor...
-> Parça 9 işlendi
-> Parça 9 işlendi
-> Parça 10 işleniyor...
-> Parça 10 işleniyor...
-> Parça 10 işlendi
-> Parça 10 işlendi
-> Parça 11 işleniyor...
-> Parça 11 işleniyor...
-> Parça 11 

KeyboardInterrupt: 